### An attempt to integrate PredPreyGrassEnv into ray rllib 2.9.3

In [ ]:
#NUMBER I
#define policies in dict with PolicySpec
#add width and height parameters to the environment

from environments.predpreygrass_env import PredPreyGrassEnv
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.utils.pre_checks.env import  check_env
from config.config_rllib import configuration
import time
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 


check_env(PredPreyGrassEnv(configuration))

env = PredPreyGrassEnv(configuration=configuration)

policy1 = PolicySpec(None,observation_space=env.observation_space, action_space=env.action_space)
policy2 = PolicySpec(None,observation_space=env.observation_space, action_space=env.action_space)

policies = { 
    "policy1": policy1,
    "policy2": policy2
}

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id.lower().startswith("predator"):
        return "policy1"
    if agent_id.lower().startswith("prey"):
        return "policy2"
    else:
        raise ValueError(f"Unexpected agent ID: {agent_id}")

config = (
    PPOConfig()
    .environment(env=PredPreyGrassEnv,env_config=configuration)
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0,log_level="ERROR")
    .training(model={
        "fcnet_hiddens" : [64, 64], 
        "_disable_preprocessor_api": False,
        "conv_filters": [[32, [8, 8], 4], [64, [4, 4], 2], [512, [1, 1], 1]] # Copilot
        }
    )
    .multi_agent(
        policies=policies,
        policy_mapping_fn=policy_mapping_fn
    )
)

algo = config.build()


env = PredPreyGrassEnv(configuration=configuration)
obs, _ = env.reset()
dones = {"__all__" : False}

#print(obs)
    
algo.stop()



In [ ]:
#NUMBER II
#define policies in dict with PolicySpec
#add width and height parameters to the environment

from environments.predpreygrass_env import PredPreyGrassEnv
from config.config_rllib import configuration

from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.utils.pre_checks.env import  check_env
from ray.rllib.policy.policy import PolicySpec
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 

env_config = {
    "no_done_at_end": True,
    "rollout_fragment_length": 1,
    "batch_mode": "complete_episodes",
    "num_envs_per_worker": 1,
    "num_workers": 0   
    }

check_env(PredPreyGrassEnv(configuration))

env = PredPreyGrassEnv(configuration=configuration)

#policy1 = PolicySpec(None,observation_space=env.observation_space, action_space=env.action_space)
#policy2 = PolicySpec(None,observation_space=env.observation_space, action_space=env.action_space)


policy1 = PolicySpec()
policy2 = PolicySpec()

policies = { 
    "policy1": policy1,
    "policy2": policy2
}

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id.lower().startswith("predator"):
        return "policy1"
    if agent_id.lower().startswith("prey"):
        return "policy2"
    else:
        raise ValueError(f"Unexpected agent ID: {agent_id}")

config = (
    PPOConfig()
    .environment(env=PredPreyGrassEnv, env_config=env_config)
    .framework("torch")
    .experimental(_enable_new_api_stack=False)
    .rollouts(
        create_env_on_local_worker=True,
        batch_mode="complete_episodes", 
        num_rollout_workers=0
    )
    .debugging(seed=0,log_level="ERROR")
    .training(model={
        "fcnet_hiddens" : [64, 64], 
        "_disable_preprocessor_api": False,
        "conv_filters": [[32, [8, 8], 4], [64, [4, 4], 2], [512, [1, 1], 1]] # Copilot
        }
    )
    .multi_agent(
        policies=policies,
        policy_mapping_fn=policy_mapping_fn
    )
    #.build()
)


algo = config.build()

obs, _ = env.reset()
dones = {"__all__" : False}

#print(obs)

action0 = algo.compute_single_action(obs["predator_0"], policy_id="policy1")
action1 = algo.compute_single_action(obs["predator_1"], policy_id="policy1")
action2 = algo.compute_single_action(obs["predator_2"], policy_id="policy1")
action3 = algo.compute_single_action(obs["prey_3"], policy_id="policy2")
action4 = algo.compute_single_action(obs["prey_4"], policy_id="policy2")
action5 = algo.compute_single_action(obs["prey_5"], policy_id="policy2")
print("action0",action0)
print("action1",action1)
print("action2",action2)
print("action3",action3)
print("action4",action4)
print("action5",action5)

algo.train()


algo.stop()



In [ ]:
#NUMBER III
#define policies in dict with PolicySpec
#add width and height parameters to the environment

from environments.predpreygrass_env import PredPreyGrassEnv
from config.config_rllib import configuration

from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.utils.pre_checks.env import  check_env
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 

env_config = {
    "no_done_at_end": True,
    "rollout_fragment_length": 1,
    "batch_mode": "complete_episodes",
    "num_envs_per_worker": 1,
    "num_workers": 0   
    }

check_env(PredPreyGrassEnv(configuration))

def env_creator(configuration):
    return PredPreyGrassEnv(configuration)  # return an env instance

register_env("pred_prey_grass", env_creator)

policy1 = PolicySpec()
policy2 = PolicySpec()

policies = { 
    "policy1": policy1,
    "policy2": policy2
}

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id.startswith("predator_"):
        return "policy1"
    if agent_id.startswith("prey_"):
        return "policy2"
    else:
        raise ValueError(f"Unexpected agent ID: {agent_id}")

config = (
    PPOConfig()
    .environment(env="pred_prey_grass", env_config=env_config)
    .framework("torch")
    .rollouts(
        create_env_on_local_worker=True,
        batch_mode="complete_episodes", 
        num_rollout_workers=0
    )
    .debugging(seed=0,log_level="ERROR")
    .training(model={
        "fcnet_hiddens" : [64, 64], 
        "_disable_preprocessor_api": False,
        "conv_filters": [[32, [8, 8], 4], [64, [4, 4], 2], [512, [1, 1], 1]] # Copilot
        }
    )
    .multi_agent(
        policies=policies,
        policy_mapping_fn=policy_mapping_fn
    )
    #.build()
)


algo = config.build()

env = PredPreyGrassEnv(configuration=configuration)
obs, _ = env.reset()

#print(obs)

action0 = algo.compute_single_action(obs["predator_0"], policy_id="policy1")
action1 = algo.compute_single_action(obs["predator_1"], policy_id="policy1")
action2 = algo.compute_single_action(obs["predator_2"], policy_id="policy1")
action3 = algo.compute_single_action(obs["prey_3"], policy_id="policy2")
action4 = algo.compute_single_action(obs["prey_4"], policy_id="policy2")
action5 = algo.compute_single_action(obs["prey_5"], policy_id="policy2")
"""
print("action0",action0)
print("action1",action1)
print("action2",action2)
print("action3",action3)
print("action4",action4)
print("action5",action5)
"""
algo.train()


algo.stop()



In [ ]:
#NUMBER IV
from environments.predpreygrass_env import PredPreyGrassEnv
from config.config_rllib import configuration

from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.utils.pre_checks.env import  check_env
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 

env_config = {
    "no_done_at_end": True,
    "rollout_fragment_length": 1,
    "batch_mode": "complete_episodes",
    "num_envs_per_worker": 1,
    "num_workers": 0   
    }

check_env(PredPreyGrassEnv(configuration))

def env_creator(configuration):
    return PredPreyGrassEnv(configuration)  # return an env instance

register_env("pred_prey_grass", env_creator)

policy1 = PolicySpec()
policy2 = PolicySpec()

policies = { 
    "policy1": policy1,
    "policy2": policy2
}

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id.lower().startswith("predator"):
        return "policy1"
    if agent_id.lower().startswith("prey"):
        return "policy2"
    else:
        raise ValueError(f"Unexpected agent ID: {agent_id}")

config = (
    PPOConfig()
    .environment(env="pred_prey_grass", env_config=env_config)
    .framework("torch")
    .rollouts(
        create_env_on_local_worker=True, 
        rollout_fragment_length=1,
        batch_mode="complete_episodes",
    )
    .debugging(seed=0,log_level="ERROR")
    .training(model={
        "fcnet_hiddens" : [64, 64], 
        "_disable_preprocessor_api": False,
        "conv_filters": [[32, [8, 8], 4], [64, [4, 4], 2], [512, [1, 1], 1]] # Copilot
        }
    )
    .multi_agent(
        policies=policies,
        policy_mapping_fn=policy_mapping_fn
    )
)


algo = config.build()

env = PredPreyGrassEnv(configuration=configuration)
obs, _ = env.reset()

#print(obs)

action0 = algo.compute_single_action(obs["predator_0"], policy_id="policy1")
action1 = algo.compute_single_action(obs["predator_1"], policy_id="policy1")
action2 = algo.compute_single_action(obs["predator_2"], policy_id="policy1")
action3 = algo.compute_single_action(obs["prey_3"], policy_id="policy2")
action4 = algo.compute_single_action(obs["prey_4"], policy_id="policy2")
action5 = algo.compute_single_action(obs["prey_5"], policy_id="policy2")
print("action0",action0)
print("action1",action1)
print("action2",action2)
print("action3",action3)
print("action4",action4)
print("action5",action5)

algo.train()


algo.stop()



In [ ]:
#define policies in dict with PolicySpec
#add width and height parameters to the environment
import ray
from ray import train, tune

from environments.predpreygrass_env import PredPreyGrassEnv
from ray.rllib.algorithms.ppo import PPOConfig
from config.config_rllib import configuration
from ray.rllib.policy.policy import PolicySpec
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 

ray.init(ignore_reinit_error=True)

env = PredPreyGrassEnv(configuration=configuration)

policy1 = PolicySpec(None,observation_space=env.observation_space, action_space=env.action_space)
policy2 = PolicySpec(None,observation_space=env.observation_space, action_space=env.action_space)

policies = { 
    "policy1": policy1,
    "policy2": policy2
}

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id[0:7] == "predator":
        return "policy1"
    elif agent_id[0:3] == "prey":
        return "policy2"

config = (
    PPOConfig()
    .environment(env=PredPreyGrassEnv,env_config=configuration)
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0,log_level="ERROR")
    .training(model={
        "fcnet_hiddens" : [64, 64], 
        "_disable_preprocessor_api": False,
        "conv_filters": [[32, [8, 8], 4], [64, [4, 4], 2], [512, [1, 1], 1]] # Copilot
        }
    )
    .multi_agent(
        policies=policies,
        policy_mapping_fn=policy_mapping_fn
    )
)


tuner = tune.Tuner(
    "PPO",
    run_config=train.RunConfig(
        stop={"episode_reward_mean": 15},
    ),
    param_space=config,
)

tuner.fit()




In [ ]:
#define policies in dict with PolicySpec
#add width and height parameters to the environment
#train the environment with the algo
#use tune for a gridsearch

import ray
from ray import train, tune

from environments.predpreygrass_env import PredPreyGrassEnv
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.utils.pre_checks.env import  check_env
from config.config_rllib import configuration
import time
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 
import time


ray.init(ignore_reinit_error=True)
                                                   
def env_creator(configuration):
    return PredPreyGrassEnv(configuration)  # return an env instance

register_env("pred_prey_grass", env_creator)

policies = { "policy1": PolicySpec(), "policy2": PolicySpec() }

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id[0:7] == "predator":
        return "policy1"
    elif agent_id[0:3] == "prey":
        return "policy2"


config = (
    PPOConfig()
    .environment(env="pred_prey_grass")
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0,log_level="ERROR")
    #.training(model={"fcnet_hiddens" : [64, 64]},lr=tune.grid_search([0.01, 0.001, 0.0001]))
    .training(model={"fcnet_hiddens" : [64, 64]})
    .multi_agent(
        policies=policies,
        policy_mapping_fn=policy_mapping_fn
    )
)


tuner = tune.Tuner(
    "PPO",
    run_config=train.RunConfig(
        stop={"episode_reward_mean": 15},
    ),
    param_space=config,
)

tuner.fit()



In [ ]:
#define policies in dict with PolicySpec
#add width and height parameters to the environment
#train the environment with the algo
#use tune for a gridsearch
#retrieving the checkpoint(s) of the trained agent

import ray
from ray import train, tune

from multi_agent_env_ray_2_9_3 import MultiAgentArena
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

import time

MAX_WIDTH = 10
MAX_HEIGHT = 10

ray.init(ignore_reinit_error=True)
                                                   
def env_creator(env_config):
    print(env_config)
    return MultiAgentArena(config=env_config, width=MAX_WIDTH, height=MAX_HEIGHT)  # return an env instance
register_env("multi_agent_arena", env_creator)

policies = { "policy1": PolicySpec(), "policy2": PolicySpec() }

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id == "agent1":
        return "policy1"
    else:
        return "policy2"


config = (
    PPOConfig()
    .environment(env="multi_agent_arena")
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0,log_level="ERROR")
    .training(model={"fcnet_hiddens" : [64, 64]},lr=tune.grid_search([0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001]))
    .multi_agent(
        policies=policies,
        policy_mapping_fn=policy_mapping_fn
    )
)


# ``Tuner.fit()`` allows setting a custom log directory (other than ``~/ray-results``)
tuner = ray.tune.Tuner(
    "PPO",
    param_space=config,
    run_config=train.RunConfig(
        stop={"episode_reward_mean": 10},
        checkpoint_config=train.CheckpointConfig(checkpoint_at_end=True),
    ),
)

results = tuner.fit()

# Get the best result based on a particular metric.
best_result = results.get_best_result(metric="episode_reward_mean", mode="max")

# Get the best checkpoint corresponding to the best result.
best_checkpoint = best_result.checkpoint
